In [1]:
import math
import os
import gc
import re
import sys
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from scipy.optimize import leastsq
from sklearn.linear_model import LinearRegression
from scipy import sparse
import numpy as np

## 一.品牌匹配

In [5]:
open_category = pd.read_csv('../tmp/train/open_category.csv')
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all = car_autohome_all.drop(['body','energy'],axis=1)
car_autohome_all = car_autohome_all.loc[:, ['brand_name','brand_slug','model_name','model_slug','detail_name','detail_slug','online_year','price_bn']]
car_autohome_all = car_autohome_all.sort_values(by=['brand_name','model_name','online_year','price_bn'])

# 汽车之家品牌
car_autohome_brand = car_autohome_all.loc[:, ['brand_name','brand_slug']].drop_duplicates(['brand_slug']).rename(columns={'brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
print('汽车之家品牌数量:', len(car_autohome_brand))

liyang_brand = open_category.loc[(open_category['parent'].isnull()), ['id','name']].rename(columns={'name':'brand_name'}).reset_index(drop=True)
print('力洋品牌数量:', len(liyang_brand))
brand = liyang_brand.merge(car_autohome_brand, how='left', on=['brand_name'])
brand = brand.sort_values(by=['brand_name'])
car_autohome_brand = car_autohome_brand.sort_values(by=['brand_name'])
car_autohome_brand.to_csv('../tmp/train/match_car_autohome_brand.csv', index=False)
brand.to_csv('../tmp/train/final_brand.csv', index=False)
brand.info()
print('请人工匹配未匹配上品牌.')

汽车之家品牌数量: 196
力洋品牌数量: 156
<class 'pandas.core.frame.DataFrame'>
Int64Index: 156 entries, 114 to 111
Data columns (total 3 columns):
id                       156 non-null int64
brand_name               156 non-null object
car_autohome_brand_id    112 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.9+ KB
请人工匹配未匹配上品牌.


## 二.车系匹配

In [28]:
brand = pd.read_csv('../tmp/train/final_brand.csv')

open_category = pd.read_csv('../tmp/train/open_category.csv')
open_model_detail = pd.read_csv('../tmp/train/open_model_detail.csv')
open_category_brand = open_category.loc[(open_category['parent'].isnull()), ['id','slug']].rename(columns={'slug':'parent'}).reset_index(drop=True)
brand = open_category_brand.merge(brand,how='left',on=['id'])
model = open_category.loc[(open_category['parent'].notnull()), ['id','name','slug','parent']].rename(columns={'name':'model_name',
                         'slug':'model_slug','id':'model_id'}).reset_index(drop=True)
model = model.merge(brand, how='left', on=['parent']).rename(columns={'id':'brand_id'})
model = model.loc[:, ['brand_id','model_id','brand_name','model_name','model_slug','car_autohome_brand_id']]
print('力洋车系数量:', len(model))

# 汽车之家车系
car_autohome_model = car_autohome_all.loc[:, ['brand_slug','model_slug','model_name']].drop_duplicates(['model_slug']).rename(columns={'model_slug':'car_autohome_model_id','brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
print('汽车之家车系数量:', len(car_autohome_model))

model = model.merge(car_autohome_model, how='left', on=['car_autohome_brand_id','model_name'])
model.loc[(model['car_autohome_brand_id'].isnull()), 'car_autohome_brand_id'] = -1
model.loc[(model['car_autohome_brand_id'] == -1), 'car_autohome_model_id'] = -1
model = model.sort_values(by=['brand_name','model_name'])
model.to_csv('../tmp/train/final_model.csv', index=False)

temp = car_autohome_all.loc[:, ['model_slug','brand_name','model_name']].drop_duplicates(['model_slug'])                                                                                                                            
temp.to_csv('../tmp/train/match_car_autohome_model.csv', index=False)
model.info()
# model.head()
# model = model.merge(brand, how='left', on=['parent']).rename(columns={'parent':'brand_slug'})

力洋车系数量: 1765
汽车之家车系数量: 1826
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1765 entries, 890 to 539
Data columns (total 7 columns):
brand_id                 1765 non-null int64
model_id                 1765 non-null int64
brand_name               1765 non-null object
model_name               1765 non-null object
model_slug               1765 non-null object
car_autohome_brand_id    1765 non-null float64
car_autohome_model_id    608 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 110.3+ KB


In [47]:
def process_model_name(df):
    text = df['model_name']
    text = text.replace(df['brand_name'],'')
    text = text.replace(' ', '')
    text = text.lower()
    return text

model = pd.read_csv('../tmp/train/final_model.csv')
part1 = model.loc[(model['car_autohome_model_id'].notnull()), :].reset_index(drop=True)
part2 = model.loc[(model['car_autohome_model_id'].isnull()), :].reset_index(drop=True)
part2['model_name'] = part2.apply(process_model_name, axis=1)
part2 = part2.drop(['car_autohome_model_id'], axis=1)
car_autohome_model = car_autohome_all.loc[:, ['brand_slug','brand_name','model_slug','model_name']].drop_duplicates(['model_slug']).rename(columns={'model_slug':'car_autohome_model_id','brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
car_autohome_model['model_name'] = car_autohome_model.apply(process_model_name, axis=1)
car_autohome_model = car_autohome_model.drop(['brand_name'], axis=1)
part2 = part2.merge(car_autohome_model, how='left', on=['car_autohome_brand_id','model_name'])
final = part1.append(part2, sort=False)
final = final.sort_values(by=['brand_name','model_name'])
final.to_csv('../tmp/train/final_model.csv', index=False)
part2.head()

,brand_id,model_id,brand_name,model_name,model_slug,car_autohome_brand_id,car_autohome_model_id
0,1851,1881,Pagani,zonda,bseries_964,61,NaN
1,1346,1203,一汽,佳宝,jiabao,110,NaN
2,1346,1295,一汽,夏利2000,xiali2000,110,NaN
3,1346,1291,一汽,夏利a+,xiali-ajia,110,NaN
4,1346,1292,一汽,夏利n3,xiali-n3,110,NaN


## 三.款型匹配

In [69]:
open_model_detail = pd.read_csv('../tmp/train/open_model_detail.csv').rename(columns={'year':'online_year','global_slug':'model_slug'})
open_model_detail = open_model_detail.drop(['volume','control'],axis=1)
final_brand = pd.read_csv('../tmp/train/final_brand.csv')
final_model = pd.read_csv('../tmp/train/final_model.csv')
final_model = final_model.loc[(final_model['car_autohome_model_id'].notnull())&(final_model['car_autohome_model_id'] != -1), :].reset_index(drop=True)
gpj_match_model = open_model_detail.loc[(open_model_detail['model_slug'].isin(list(set(final_model.model_slug.values)))), :]
gpj_match_model = gpj_match_model.merge(final_model.loc[:,['model_slug','car_autohome_brand_id','car_autohome_model_id']],how='left',on=['model_slug'])
print('匹配上的车系', len(final_model))

car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all = car_autohome_all.drop(['manufacturer','body','energy'], axis=1)
print('汽车之家车系总量:',len(list(set(car_autohome_all.model_slug.values))))

# 汽车之家补充的车系
supplement_model = car_autohome_all.loc[~(car_autohome_all['model_slug'].isin(list(set(final_model.car_autohome_model_id.values)))), :].reset_index(drop=True)
match_model = car_autohome_all.loc[(car_autohome_all['model_slug'].isin(list(set(final_model.car_autohome_model_id.values)))), :].reset_index(drop=True)
match_model = match_model.rename(columns={'model_slug':'car_autohome_model_id'}).loc[:, ['car_autohome_model_id','detail_name','price_bn','online_year']]
print('汽车之家补充的车系量:', len(list(set(supplement_model.model_slug.values))))
print('汽车之家匹配的车系量:', len(list(set(match_mode.model_slug.values))))

final_detail = gpj_match_model.merge(match_model,how='left',on=['car_autohome_model_id','online_year','price_bn'])
final_detail = final_detail.drop_duplicates(['car_autohome_model_id','online_year','price_bn','detail_model'])
# car_autohome_all.head()

匹配上的车系 1513
汽车之家车系总量: 1826
汽车之家补充的车系量: 313
汽车之家匹配的车系量: 1513


In [58]:
gpj_match_model.head()

,id,price_bn,model_slug,online_year,detail_model,model_detail_slug,car_autohome_brand_id,car_autohome_model_id
0,96,81.80,audi-a7,2012,2012款 3.0TFSI S-Tronic quattro 舒适型,model_25023_cs,33,740.0
1,97,95.80,audi-a7,2012,2012款 3.0TFSI S-Tronic quattro 豪华型,model_15554_cs,33,740.0
2,120,48.98,i-audi-a4,2013,2013款 2.0T 40 TFSI allroad quattro 舒适版,model_28145_cs,33,471.0
3,121,51.98,i-audi-a4,2013,2013款 2.0T 40 TFSI allroad quattro 豪华版,model_26127_cs,33,471.0
4,161,345.00,astonmartin-db9,2011,2011款 6.0L 自动 Touchtronic Volante,model_25573_cs,35,266.0


In [63]:
match_model.head()

,car_autohome_model_id,detail_name,price_bn,online_year
0,16,2005款 CIF 基本型,9.58,2005
1,16,2005款 GIF 前卫豪华型,8.78,2005
2,16,2008款 CIF 春天型,8.98,2008
3,16,2008款 CIF+AT 春天型,10.18,2008
4,16,2004款 GDF 豪华型,10.13,2004


In [70]:
final_detail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34507 entries, 0 to 40124
Data columns (total 9 columns):
id                       34507 non-null int64
price_bn                 34507 non-null float64
model_slug               34507 non-null object
online_year              34507 non-null int64
detail_model             34507 non-null object
model_detail_slug        34507 non-null object
car_autohome_brand_id    34507 non-null int64
car_autohome_model_id    34507 non-null float64
detail_name              24680 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 2.6+ MB


In [66]:
gpj_match_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34546 entries, 0 to 34545
Data columns (total 8 columns):
id                       34546 non-null int64
price_bn                 34546 non-null float64
model_slug               34546 non-null object
online_year              34546 non-null int64
detail_model             34546 non-null object
model_detail_slug        34546 non-null object
car_autohome_brand_id    34546 non-null int64
car_autohome_model_id    34546 non-null float64
dtypes: float64(2), int64(3), object(3)
memory usage: 2.4+ MB
